In [ ]:
import os
import time
import shutil
import pdfplumber
from API-YoungsModulus import call_deepseek_llm
from contains_keywords-youngs import contains_keywords
from Clean import ParagraphParser  # 用这个过滤

# 输入文件夹路径
input_folder = r"Your_split_pdfs_Path"
processed_folder = os.path.join(input_folder, "processed_pdfs")     # 已处理成功的
wasted_pdfs = os.path.join(input_folder, "wasted_pdfs")         # 什么都没有的文献
os.makedirs(processed_folder, exist_ok=True)
os.makedirs(wasted_pdfs, exist_ok=True)

# 关键词字典（聚合物 以及 力学性能）
keyword_dict = {
    "Polymers": [
        # 通用聚合物及共聚物
        "polymer", "copolymer", "blend", "biopolymer",
        # 常见商用或合成聚合物
        "PLA", "polylactic acid", "PCL", "polycaprolactone",
        "PET", "polyethylene terephthalate", "PMMA", "polymethyl methacrylate",
        "PU", "polyurethane", "PA", "polyamide", "nylon",
        "PVC", "polyvinyl chloride", "PVA", "polyvinyl alcohol",
        "PAN", "polyacrylonitrile", "PVP", "polyvinylpyrrolidone",
        "PDMS", "polydimethylsiloxane", "PC", "polycarbonate",
        "PBS", "polybutylene succinate", "PGA", "poly(γ-glutamic acid)",
        "PE", "polyethylene", "PP", "polypropylene", "polyester",
        # 生物基/天然聚合物
        "GelMA", "gelatin methacrylate", "gelatin", "collagen",
        "chitosan", "alginate", "sodium alginate", "cellulose",
        "nanocellulose", "pectin", "lignin", "starch", "hyaluronic acid",
        "silk fibroin", "polyethylene glycol", "polydopamine",
        "polyacrylamide"
    ],
    "Additives or Modifiers": [
        # 基础功能添加剂
        "additive", "modifier", "plasticizer", "compatibilizer", "filler", "blend",
        "hybrid", "nanocomposite", "composite",
        # 无机纳米材料
        "nanoparticle", "nanofiller", "nanoclay", "TiO2", "SiO2", "ZnO", "CaCO3",
        "clay", "montmorillonite", "halloysite", "bentonite",
        # 有机/高分子共混物
        "PBAT", "PEG", "PHA", "PBSA", "PPC", "EVA", "PLA-g-MA",
        # 生物基/天然材料
        "cellulose nanocrystal", "microcrystalline cellulose", "hemicellulose",
        "soy protein", "wheat bran", "rice husk",
        # 碳基材料
        "CNT", "carbon nanotube", "carbon black", "graphene", "graphene oxide",
        "reduced graphene oxide",
        # 纤维及增强材料
        "fiber", "natural fiber", "glass fiber", "bamboo fiber", "hemp fiber",
        "basalt fiber", "jute fiber", "kenaf fiber",
        # 增塑剂
        "glycerol", "triacetin", "citrate", "ATBC", "TEC", "tributyl citrate",
        "polyethylene glycol",
        # 共混及相容化
        "blending", "blended", "copolymerized", "reactive compatibilization",
        "immiscible", "miscible",
        # 其他添加剂
        "antioxidant", "nucleating agent", "chain extender", "crosslinker",
        "UV stabilizer", "thermal stabilizer", "fire retardant", "flame retardant"
    ],
    "mechanical properties":
    [
    "tensile strength", "breaking strength", "tensile properties",
    "elongation at break", "breaking elongation",
    "young's modulus", "tensile modulus",
    "flexural modulus", "bending modulus", "flexural stiffness",
    "impact strength", "impact toughness",
    "stress-strain", "mechanical behavior",
    "hardness", "shore hardness", "rockwell", "durometer",
    ],
    "Glass Transition": ["glass transition", "Tg"],
    "Melting Point": ["melting point", "melting temperature", "Tm"]
    
}
def extract_pdf_with_layout(pdf_path):
    """提取文本，不提表格，避免 Ascii85 错误"""
    full_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            try:
                text = page.extract_text() or ""
                full_text.append(f"\n=== Page {page.page_number} ===\n{text}")
            except Exception as e:
                full_text.append(f"\n⚠️ 第 {page.page_number} 页提取失败：{e}")
    return "\n".join(full_text)


def process_pdf(pdf_path):
    filename = os.path.basename(pdf_path)
    out_dir = os.path.join(os.path.dirname(pdf_path), "extracted_output")
    os.makedirs(out_dir, exist_ok=True)
    txt_path = os.path.join(out_dir, filename.replace(".pdf", ".txt"))

    if os.path.exists(txt_path):
        print(f"⏩ 已处理过，跳过：{filename}")
        shutil.move(pdf_path, os.path.join(processed_folder, filename))
        return

    try:
        raw = extract_pdf_with_layout(pdf_path)
    except Exception as e:
        print(f"❌ {filename} 提取失败：{e}")
        return

    parser = ParagraphParser(name=filename, debug=False)
    parser.parse(raw)
    if not parser.is_valid():
        return
    text = parser.text.strip()

    # Step 1: 判断是否含 "聚合物"
    found_polymers, _, matched_polymer_words = contains_keywords(text, {"Polymers": keyword_dict["Polymers"]})
    if not found_polymers:
        print(f"⏩ 路径为 {pdf_path} 的 {filename} 不含 Polymers 关键词，移动到 wasted_pdfs")
        shutil.move(pdf_path, os.path.join(wasted_pdfs, filename))
        return
    else:
        print(f"📌 路径为 {pdf_path} 的 {filename} 命中 Polymers 关键词：{', '.join(matched_polymer_words.get('Polymers', []))}")

    # Step 2: 判断是否包含力学性能关键词
    mech_dict = {"mechanical properties": keyword_dict["mechanical properties"]}
    found_mech, matched_categories, matched_words = contains_keywords(text, mech_dict)
    if not found_mech:
        print(f"⏩ 路径为 {pdf_path} 的 {filename} 虽含 Polymers 关键词，但未检测到任何 力学性能 关键词，移动到 wasted_pdfs")
        shutil.move(pdf_path, os.path.join(wasted_pdfs, filename))
        return
    else:
        print(f"📈 路径为 {pdf_path} 的 {filename} 命中 力学性能 关键词：{', '.join(matched_words.get('mechanical properties', []))}")

    for cat, words in matched_words.items():
        print(f"    - {cat}：{', '.join(words)}")

    prompt = f"📃 相关参数的文本如下：\n{text}"
    try:
        ans = call_deepseek_llm(prompt)
        if not ans or not ans.strip():
            print(f"⚠️ 路径为 {pdf_path} 的 {filename} 无API响应，跳过")
            return
        content = f"🧾 提取结果：\n{ans.strip()}\n\n"
    except Exception as e:
        print(f"❌ 路径为 {pdf_path} 的 {filename} 处理出错：{e}")
        return

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(content)
    shutil.move(pdf_path, os.path.join(processed_folder, filename))
    print(f"✅ 提取完成：{filename}")
    print(f"📂 原路径：{pdf_path}")
    print(f"📄 写入路径：{txt_path}")


# 遍历处理所有 PDF（方法一：过滤子目录）
for root, dirs, files in os.walk(input_folder):
    # 过滤不需递归的子目录
    dirs[:] = [d for d in dirs if d not in (
        os.path.basename(processed_folder),
        os.path.basename(wasted_pdfs),
        'extracted_output'
    )]
    for fn in files:
        if fn.lower().endswith('.pdf') and not fn.startswith('._'):
            process_pdf(os.path.join(root, fn))

